In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter

# 读取xlsx文件
data = pd.read_excel(r'C:\Users\RIC_ZX\Desktop\papers\ZLW\(申请号去重后）美国和欧盟 已筛选TAC_(poly butylene adipate-co-terephthalate) OR TAC_(PBAT).xlsx')

In [2]:
data['IPC主分类号'] = data['IPC主分类号'].str[:4]
data

,序号,公开(公告)号,标题,法律状态/事件,申请日,发明人,申请号,权利要求,简单同族,简单同族成员数量,...,优先权国家/地区,被引用专利,被引用专利数量,引用专利,引用专利数量,非专利引文数量,许可类型,5年内被引用次数,诉讼案件数,质押人
0,1057,US675631A,Thio-urea compound and process of making same.,期限届满,1900-12-31,ADOLF ISRAEL | RICHARD KOTHE,US1900041628,Having now described our invention and in 20 w...,US675631A,1,...,-,US5144064A,NaN,-,0,0,-,0,0,-
1,1054,US5599858A,Aliphatic-aromatic copolyesters and cellulose ...,期限届满,1995-04-26,"BUCHANAN, CHARLES M. | GARDNER, ROBERT M. | WO...",US08/427944,We claim:_x000D_\n1. One or more fibers prepar...,AT199383T | AT349476T | ATE150058T1 | ATE19938...,46,...,-,AU752805B2 | CN100398306C | CN100558784C | CN1...,NaN,DE2703417A1 | DE2829624A1 | EP0387167A1 | EP03...,50,14,-,38,0,-
2,10,EP1685201A1,Use of an aqueous dispersion of at least one b...,撤回-视为撤回,2004-10-29,"MENTINK, LÉON | BOUVIER, FRÉDÉRIC | CLABAUX, P...",EP2004805352,"REVENDICATIONS_x000D_\n1. Utilisation, pour la...",CA2546107A1 | EP1685201A1 | FR2862310A1 | FR28...,8,...,FR | FR,-,NaN,-,0,0,-,0,0,-
3,650,US20070021515A1,Expandable starch-based beads and method of ma...,撤回-未指定类型,2005-07-19,"GLENN, GREGORY M. | HODSON, SIMON K.",US11/184614,"1. An expandable starch-based composition, the...",US20070021515A1,1,...,-,CN111607126A | CN111607126B | JP2013530301A | ...,NaN,US3850863A | US3867324A | US3949145A | US40768...,75,0,-,8,0,-
4,651,US20070021534A1,Fiber-reinforced starch-based compositions and...,未缴年费,2005-07-19,"GLENN, GREGORY M. | HODSON, SIMON K.",US11/184128,1. A starch-based and fiber-reinforced composi...,US20070021534A1 | US7989524B2,1,...,-,CN101730622A | CN101962814A | CN101962814B | C...,NaN,US20020006989A1 | US20020071947A1 | US20030100...,99,0,-,21,0,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,999,US20230123482A1,"Recyclable, Biodegradable, and Industrially Co...",实质审查,2022-12-19,"FALKEN, ROBERT",US18/068115,"1. A process for producing a flexible foam, th...",AU2021379913A1 | CN116615323A | EP4244040A1 | ...,8,...,US,-,NaN,-,0,0,-,0,0,-
445,1018,US20230212537A1,Novel esterases and uses thereof,公开,2022-12-22,"DAVID, BENOÎT | ANDRE, ISABELLE | BEN KHALED, ...",US18/145073,1. An esterase which (i) has at least 95% iden...,CA3107517A1 | CN112654701A | EP3830254A1 | IN2...,10,...,EP,-,NaN,-,0,0,-,0,0,-
446,1017,US20230208143A1,System and method for smoothing photovoltaic g...,公开,2022-12-22,"AREDES, MAURICIO | BARBOSA ROLIM, LUÍS GUILHER...",US18/087199,1. A system for smoothing photovoltaic generat...,BR102021026269A2 | US20230208143A1,2,...,BR,-,NaN,-,0,0,-,0,0,-
447,1001,US20230127747A1,Biodegradable and Industrially Compostable Inj...,实质审查 | 权利转移,2022-12-23,"FALKEN, ROBERT",US18/087934,1-20. (canceled)_x000D_\n21. A method for manu...,AU2019272664A1 | BR112020023590A2 | CN11251277...,12,...,US,-,NaN,US20100198133A1 | US20130154143A1,2,0,-,0,0,-


In [16]:
# 构建边列表
edges = []
for row in data.iterrows():
    _, patent = row
    cited_patents = str(patent['引用专利']).split(' | ')
    for cited_patent in cited_patents:
        if cited_patent != '-':
            edges.append((patent['公开(公告)号'], cited_patent))

# 统计第二列中每个字符串的数量
string_counts = Counter(edge[0] for edge in edges)

# 仅保留字符串数量大于等于2的行
filtered_edges = [edge for edge in edges if string_counts[edge[0]] >= 15]

# 创建有向图
G = nx.DiGraph()
G.add_edges_from(filtered_edges)
G_patent = G#.reverse()
# 生成带权邻接矩阵
adj_matrix = nx.to_pandas_adjacency(G_patent, weight='weight')

# 保存邻接矩阵到CSV文件
#adj_matrix.to_excel("专利p被引用网络带权邻接矩阵.xlsx")

In [17]:
adj_matrix.to_excel("专利引用邻接矩阵10.xlsx")

In [12]:
# 计算每个节点的共引值
import itertools
co_citation_counts = Counter()
for cited_patent, group in itertools.groupby(sorted(filtered_edges, key=lambda x: x[1]), key=lambda x: x[1]):
    citing_patents = [edge[0] for edge in group]
    for i, j in itertools.combinations(citing_patents, 2):
        co_citation_counts[i] += 1
        co_citation_counts[j] += 1

# 将共引值保存到DataFrame
co_citation_df = pd.DataFrame(co_citation_counts.items(), columns=['专利号', '共引值'])

# 保存共引值到Excel文件
co_citation_df

,专利号,共引值
0,US11674014B2,732
1,US11674018B2,732
2,US11111363B2,593
3,US11149144B2,620
4,US11359088B2,691
...,...,...
102,US11142371B2,2
103,EP2432830B1,5
104,US8852747B2,34
105,US11746174B2,1


In [13]:
co_citation_df.to_excel("专利共引值10.xlsx", index=False)

In [4]:
clustering_coefficients = nx.clustering(G)  # 计算每个节点的聚类系数
# 如果需要，可以打印每个节点的聚类系数
for node, cc in clustering_coefficients.items():
    print(f"Node {node}: Clustering Coefficient = {cc}")

Node US5599858A: Clustering Coefficient = 0
Node AU752805B2: Clustering Coefficient = 0
Node CN100398306C: Clustering Coefficient = 0
Node CN100558784C: Clustering Coefficient = 0
Node CN101356306A: Clustering Coefficient = 0
Node CN101563392A: Clustering Coefficient = 0
Node CN101573481A: Clustering Coefficient = 0
Node CN101573482A: Clustering Coefficient = 0
Node CN101778897A: Clustering Coefficient = 0
Node CN101778897B: Clustering Coefficient = 0
Node CN101784711A: Clustering Coefficient = 0
Node CN102066467A: Clustering Coefficient = 0
Node CN102066467B: Clustering Coefficient = 0
Node CN102076744A: Clustering Coefficient = 0
Node CN102257068A: Clustering Coefficient = 0
Node CN102695748A: Clustering Coefficient = 0
Node CN102704036A: Clustering Coefficient = 0
Node CN102704036B: Clustering Coefficient = 0
Node CN102877160A: Clustering Coefficient = 0
Node CN102877160B: Clustering Coefficient = 0
Node CN102888671A: Clustering Coefficient = 0
Node CN102888671B: Clustering Coeffici

In [5]:
average_clustering = nx.average_clustering(G)  # 计算整个图的平均聚类系数
print(f"Average Clustering Coefficient = {average_clustering}")

Average Clustering Coefficient = 0.002670689113021839


In [6]:
betweenness_centrality = nx.betweenness_centrality(G_patent)
closeness_centrality = nx.closeness_centrality(G_patent)

# 将指标结果转换为DataFrame
bc_df = pd.DataFrame(list(betweenness_centrality.items()), columns=['Node', 'Betweenness Centrality'])
cc_df = pd.DataFrame(list(closeness_centrality.items()), columns=['Node', 'Closeness Centrality'])

# 保存指标结果为CSV文件
bc_df.to_excel("betweenness_centrality_patent.xlsx", index=False)
cc_df.to_excel("closeness_centrality_patent.xlsx", index=False)

In [7]:
import pandas as pd

# 计算每个节点的入度和出度
in_degrees = G_patent.in_degree()
out_degrees = G_patent.out_degree()
# 创建 DataFrame 存储节点、入度和出度信息
node_list = list(G_patent.nodes())
degree_data = {'Node': node_list, 'In-Degree': [in_degrees[node] for node in node_list], 'Out-Degree': [out_degrees[node] for node in node_list]}
degree_df = pd.DataFrame(degree_data)

# 保存 DataFrame 到 Excel 文件
degree_df.to_excel("节点度信息.xlsx", index=False)

In [8]:

import community

# 使用Louvain算法检测社区
partition = community.best_partition(G)

# 将社区信息添加到节点属性中
for node, community_id in partition.items():
    G.nodes[node]['community'] = community_id

# 将节点社区信息保存到CSV文件
community_data = pd.DataFrame.from_dict(partition, orient='index', columns=['community'])
community_data.index.name = 'patent_id'
community_data.to_csv('专利社区信息.csv')

# 查看每个社区的节点数量
community_sizes = pd.Series(community_data['community'].value_counts(), name='nodes_count')
community_sizes.index.name = 'community_id'
print(community_sizes)

TypeError: Bad graph type, use only non directed graph

In [37]:
import numpy as np
df = data[['公开(公告)号','被引用专利']]
df

,公开(公告)号,被引用专利
0,US675631A,US5144064A
1,US5599858A,AU752805B2 | CN100398306C | CN100558784C | CN1...
2,EP1685201A1,-
3,US20070021515A1,CN111607126A | CN111607126B | JP2013530301A | ...
4,US20070021534A1,CN101730622A | CN101962814A | CN101962814B | C...
...,...,...
444,US20230123482A1,-
445,US20230212537A1,-
446,US20230208143A1,-
447,US20230127747A1,-


In [38]:
df['被引用专利'] = df['被引用专利'].replace('-', np.nan)

C:\Users\RIC_ZX\AppData\Local\Temp\ipykernel_42012\1262573493.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['被引用专利'] = df['被引用专利'].replace('-', np.nan)


In [39]:
df = df.dropna()

In [40]:
df

,公开(公告)号,被引用专利
0,US675631A,US5144064A
1,US5599858A,AU752805B2 | CN100398306C | CN100558784C | CN1...
3,US20070021515A1,CN111607126A | CN111607126B | JP2013530301A | ...
4,US20070021534A1,CN101730622A | CN101962814A | CN101962814B | C...
5,US20060043629A1,CN101302347B | CN102159637A | CN102239210A | C...
...,...,...
411,US20220267936A1,US20230031424A1
420,US20220135791A1,CN115558139A
427,US20220227949A1,US11674014B2 | US11674018B2
429,US20220251735A1,DE202022105333U1 | WO2023135258A1
